# Simple RAG system for zendesk articles based on ollama
Assumes documents were downloaded with [this script](./download.ipynb)

In [ ]:
%pip install langchain

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import pickle
ollama_host="http://ollama:11434"

In [3]:
llm = Ollama(model="llama3",base_url=ollama_host)

def getEmbeddings():
    cache = './data/cache/articleembeddings_zdollama.pkl'
    try:
        with open(cache,"rb") as inp:
            return pickle.load(inp)
    except IOError:
        loader = TextLoader("./data/samplearticle.txt",encoding="utf-8")
        data=loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        data = text_splitter.split_documents(data)
        embeddings = OllamaEmbeddings(base_url=ollama_host)
        vectorstore = FAISS.from_documents(data, embedding=embeddings)
        with open(cache,"wb") as out:
            pickle.dump(vectorstore,out,pickle.DEFAULT_PROTOCOL)
        return vectorstore
vectorstore = getEmbeddings()

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [4]:
query = "Why would I use a transport of copies?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/workspaces/ragtest/.condaenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Based on the provided context, it seems that the topic of discussion is around using transports of copies (TOCs) in SAP ActiveControl. From the information provided, it appears that TOCs can be created at different levels (transport, task, or project) and can have a different transport description than the original source transport.\n\nThe context also mentions that before ActiveControl 8.0, TOCs were given a transport description equal to the business task subject, which could cause confusion in scenarios where multiple different source transports were linked to the same business task. To address this issue, the new TOC capability allows for the creation of TOCs with a different transport description than the original source transport.\n\nAs for why someone would use a transport of copies, it seems that the answer is not explicitly provided in the context. However, based on the information provided, it can be inferred that TOCs can be useful in scenarios where multiple transports are